---
title: Graphs
order: 5
---

This post implements (and improves) code in chapter 14 from the textbook Introduction to Computation and Programming Using Python, With Application to Computational Modeling and Understanding Data, third edition, by John V. Guttag, 2020. 

In [6]:
class Node:
    """Basic node to be used in graphs."""

    def __init__(self, name):
        """Assumes name is a string."""
        self._name = name

    def get_name(self):
        """Returns node name."""
        return self._name

    def __str__(self):
        """Represents node by its name."""
        return self._name

    def __eq__(self, other):
        """Two nodes are equal if their names are equal."""
        return self._name == other._name

I decided to implement edges as two different classes that don't inherit from each other, because I think one isn't a special case of another. I mean, perhaps it is, but methods on both will differ so substantially, that it doesn't make sense to me to have them be connected. This is because of the substitution principle: A code that works on a superclass must work on a subclass. But if a code works on bidirectional edges (perhaps exploiting symmetry) it may not work on unidirectional edges (where the symmetry doesn't exist). So, if anything, bidirectional edge must inherit from unidirectional edge. But that doesn't work either, because unidirectional edge logically must have the methods get_source and get_destination, but bidirectional edge cannot have these methods. bidirectional edges don't have sources or destination because both nodes could work as such. And which node should be returned by a bidirectional edge's get_source method? There is no clear answer, so I conclude both types of edges cannot inherit from each other in any way. The only solution would be to completely discard bidirectional edges and only work with directional ones - after all, we can build undirected graph with directed edges - but that would prevent me from having this fun :). Unfortunately, a lot of code is then duplicated.

In [ ]:
class Edge:
    """Basic unweighted bidirectional edge."""

    def __init__(self, node1, node2):
        """Assumes node1 and node2 are Nodes."""
        self._node1 = node1
        self._node2 = node2

    def get_nodes(self):
        """Returns both nodes as a tuple."""
        return self._node1, self._node2

    def __str__(self):
        """Textual representation of the edge."""
        node1, node2 = self.get_nodes()
        return f"{node1.get_name()} <----> {node2.get_name()}"

    def __eq__(self, other):
        """Two Edges are equal if they have the same nodes (order doesn't matter)."""
        return set(self.get_nodes()) == set(other.get_nodes())

In [ ]:
class WeightedEdge(Edge):
    """Weighted bidirectional edge."""

    def __init__(self, node1, node2, weight=1):
        "Assumes node1 and node2 are Nodes."
        super().__init__(node1, node2)
        self._weight = weight

    def get_weight(self):
        """Returns weight of the edge."""
        return self._weight

    def __str__(self):
        node1, node2 = self.get_nodes()
        return f"{node1.get_name()} <--({self.get_weight()})--> {node2.get_name()}"

    def __eq__(self, other):
        weights_equal = self.get_weight() == other.get_weight()
        return super().__eq__(other) and weights_equal

In [ ]:
class DirectedEdge:
    """Basic unweighted unidirectional edge."""

    def __init__(self, src, dest):
        """Assumes src and dest are Nodes."""
        self._src = src
        self._dest = dest

    def get_src(self):
        """Returns src."""
        return self._src

    def get_dest(self):
        """Returns dest."""
        return self._dest

    def __str__(self):
        """Textual representation of the edge."""
        return f"{self._src.get_name()} ----> {self._dest.get_name()}"

    def __eq__(self, other):
        """Two Edges are equal if both source and destinations are the same."""
        sources_equal = self.get_src() == other.get_src()
        destinations_equal = self.get_dest() == other.get_dest()
        return sources_equal and destinations_equal

In [10]:
class WeightedDirectedEdge(DirectedEdge):
    """Weighted unidirectional edge."""

    def __init__(self, src, dest, weight=1):
        """Assumes src and dest Nodes, weight a number."""
        super().__init__(src, dest)
        self._weight = weight

    def get_weight(self):
        """Returns weight of the edge."""
        return self._weight

    def __str__(self):
        """Textual representation of the edge."""
        src, dest = self.get_src(), self.get_dest()
        return f"{src.get_name()} --({self.get_weight()})--> {dest.get_name()}"

    def __eq__(self, other):
        weights_equal = self.get_weight() == other.get_weight()
        return super().__eq__(other) and weights_equal

In [ ]:
class C:

    def __init__(self):
        self.x = 1

    def f(self):
        print(self.x)


class B(C):
    def __init__(self):
        self.x = 2


B().f()

2


So perhaps the inheritance schema can be as follows:

- BaseEdge <- BaseWeightedEdge <- WeightedEdge
- ___________________________________<- WeightedDirectionalEdge
- ___________<- Edge
- ___________<- DirectionalEdge


BaseEdge would implement \_\_str\_\_ (with empty separator string), and attributes node1 and node2, BaseWeightedEdge would implement get_weight and perhaps \_\_eq\_\_